In [1]:
# 12.Problem Statement
# Read the following data set:
# https://archive.ics.uci.edu/ml/machine-learning-databases/adult/
# Rename the columns as per the description from this file:
# https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
# Task:
# Create a sql db from adult dataset and name it sqladb
# 1. Select 10 records from the adult sqladb
# 2. Show me the average hours per week of all men who are working in private sector
# 3. Show me the frequency table for education, occupation and relationship, separately
# 4. Are there any people who are married, working in private sector and having a mastersdegree
# 5. What is the average, minimum and maximum age group for people working in different sectors
# 6. Calculate age distribution by country
# 7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'

In [2]:
#Load the packages
import numpy as np
import pandas as pd
import os

import sqlite3 as db
from pandasql import sqldf

import sqlalchemy
from sqlalchemy import create_engine

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

#Set the path for future Reference
path="C:\\Users\\HP\\Desktop\Learning\\DataScience\\03 DataFiles"
os.chdir(path)

#Display formating 
from prettytable import PrettyTable
from prettytable import from_db_cursor

x = PrettyTable()

In [3]:
#Creating the DB
conn = db.connect('sqladb.sqlite')
cursor = conn.cursor()
print("Opened database successfully")
conn.commit()

Opened database successfully


In [4]:
#Table Creation
cursor.execute('''CREATE TABLE IF NOT EXISTS AdultTable 
               (Age INT(2), 
                Workclass VARCHAR(255), 
                fnlwgt INT(5), 
                Education VARCHAR(255),
                Edunum INT(5), 
                Maritalstatus VARCHAR(255),
                Occupation VARCHAR(255), 
                Familyrel VARCHAR(255), 
                Race VARCHAR(255), Sex VARCHAR(255), 
                Capitalgain INT(20), 
                Capitalolss INT(20), 
                Hourspweek INT(20), 
                Nativecountry VARCHAR(255), 
               Greatethan50K VARCHAR(255));''')
cursor.close()

In [5]:
#Temp table created from the csv file
conn = db.connect('sqladb.sqlite')
cursor = conn.cursor()

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data")

cursor.execute('''DROP TABLE IF EXISTS  AdultTemp''')
df.to_sql('AdultTemp', conn, if_exists='replace', index=False) # - writes the pd.df to SQLIte DB
pd.read_sql('select * from AdultTemp', conn)
conn.commit()

conn.close()

C:\Users\HP\Anaconda3\lib\site-packages\pandas\core\generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [6]:
#Insert data to the adultable which is created
conn = db.connect('sqladb.sqlite')
cursor = conn.cursor()

#Load data from sample.csv to table
stmt="INSERT INTO AdultTable SELECT * FROM AdultTemp "
#stmt ="DELETE FROM AdultTable "
cursor.execute(stmt)
# Commit is important to reflect the changes
conn.commit()
cursor.execute('''DROP TABLE IF EXISTS  AdultTemp''')
conn.commit()

                 

In [7]:
conn = db.connect('sqladb.sqlite')
cursor = conn.cursor()
# 1) Select 10 rows from table
cursor.execute("SELECT * FROM AdultTable LIMIT 10")
myresult = cursor.fetchall()
for x in myresult:
    print(x)

(50, ' Self-emp-not-inc', 83311, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K')
(38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', 0, 0, 40, ' United-States', ' <=50K')
(53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husband', ' Black', ' Male', 0, 0, 40, ' United-States', ' <=50K')
(28, ' Private', 338409, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' Black', ' Female', 0, 0, 40, ' Cuba', ' <=50K')
(37, ' Private', 284582, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' White', ' Female', 0, 0, 40, ' United-States', ' <=50K')
(49, ' Private', 160187, ' 9th', 5, ' Married-spouse-absent', ' Other-service', ' Not-in-family', ' Black', ' Female', 0, 0, 16, ' Jamaica', ' <=50K')
(52, ' Self-emp-not-inc', 209642, ' HS-grad', 9, ' Married-civ-spouse', ' Exec-managerial

In [8]:
#1. Select 10 records from the adult sqladb
#with the formated display
con = db.connect('sqladb.sqlite')
with con:
    cur = con.cursor()    
    cur.execute("SELECT * FROM AdultTable LIMIT 10")   
    x = from_db_cursor(cur) 
print(x.get_string(title="Adult Table"))
#print(x)

#print(x.get_string(fields=["Age", "Workclass","Maritalstatus"]))


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                      Adult Table                                                                                                      |
+-----+-------------------+--------+---------------+--------+------------------------+--------------------+----------------+--------+---------+-------------+-------------+------------+----------------+---------------+
| Age |     Workclass     | fnlwgt |   Education   | Edunum |     Maritalstatus      |     Occupation     |   Familyrel    |  Race  |   Sex   | Capitalgain | Capitalolss | Hourspweek | Nativecountry  | Greatethan50K |
+-----+-------------------+--------+---------------+--------+------------------------+--------------------+----------------+----

In [9]:
# 2) Average hours per week for all man who are working in private sector
stmt = "SELECT AVG(Hourspweek) AvgHourspweek FROM AdultTable WHERE Workclass = ' Private' AND Sex= ' Male' "
print(stmt)
cursor.execute(stmt)
myresult = cursor.fetchall()
for x in myresult:
    print(x)


SELECT AVG(Hourspweek) AvgHourspweek FROM AdultTable WHERE Workclass = ' Private' AND Sex= ' Male' 
(42.22122591006424,)


In [10]:
# 2) Average hours per week for all man who are working in private sector
#with formated display
con = db.connect('sqladb.sqlite')
with con:
    cur = con.cursor()    
    cur.execute("SELECT Workclass,Sex,AVG(Hourspweek) as 'Avg-per-week' FROM AdultTable  WHERE (Workclass = ' Private' AND Sex= ' Male') ")   
                #WHERE (Workclass = ' Private' AND Sex= ' Male') GROUP by Workclass ")   
    x = from_db_cursor(cur) 
print(x.get_string(title="Avg Hrs/week by Workclass Private"))
#print(x)

#print(x.get_string(fields=["Workclass","Sex","Avg-per-week"]))

+---------------------------------------+
|   Avg Hrs/week by Workclass Private   |
+-----------+-------+-------------------+
| Workclass |  Sex  |    Avg-per-week   |
+-----------+-------+-------------------+
|   Private |  Male | 42.22122591006424 |
+-----------+-------+-------------------+


In [11]:
# Coutn of male workign in private
stmt = "SELECT count(*) FROM AdultTable WHERE Workclass = ' Private' AND  Sex =' Male'"
print(stmt)
cursor.execute(stmt)
myresult = cursor.fetchall()
for x in myresult:
    print(x)
con = db.connect('sqladb.sqlite')

SELECT count(*) FROM AdultTable WHERE Workclass = ' Private' AND  Sex =' Male'
(14944,)


In [12]:
# Coutn of male workign in private
#with formating
con = db.connect('sqladb.sqlite')
with con:
    cur = con.cursor()    
    cur.execute("SELECT count(*) as CNT FROM AdultTable WHERE Workclass = ' Private' AND  Sex =' Male' ")   
    x = from_db_cursor(cur) 
print(x.get_string(title="Count of Male Working in Private"))

+----------------------+
| Count of Male Working in Private |
+----------------------+
|         CNT          |
+----------------------+
|        14944         |
+----------------------+


In [13]:
##3. Show me the frequency table for education, occupation and relationship, separately
# 3) Frequency tables
#    a) education
#cursor.execute("SELECT Education , COUNT(Education) count FROM AdultTable GROUP BY Education ")
#myresult = cursor.fetchall()
#for x in myresult:
#    print(x)
#conn.close()

con = db.connect('sqladb.sqlite')
with con:
    cur = con.cursor()    
    cur.execute("SELECT Education , COUNT(Education) Count FROM AdultTable GROUP BY Education ")   
    x = from_db_cursor(cur)
    
print(x.get_string(title="Adult Data Group by education"))
#print(x.get_string(fields=["Education","Count"]))

+------------------------------+
| Adult Data Group by education |
+--------------------+---------+
|     Education      |  Count  |
+--------------------+---------+
|        10th        |   933   |
|        11th        |   1175  |
|        12th        |   433   |
|       1st-4th      |   168   |
|       5th-6th      |   333   |
|       7th-8th      |   646   |
|         9th        |   514   |
|     Assoc-acdm     |   1067  |
|      Assoc-voc     |   1382  |
|      Bachelors     |   5354  |
|      Doctorate     |   413   |
|       HS-grad      |  10501  |
|       Masters      |   1723  |
|      Preschool     |    51   |
|     Prof-school    |   576   |
|    Some-college    |   7291  |
+--------------------+---------+


In [14]:
##3. Show me the frequency table for education, occupation and relationship, separately

#    b) occupation
#cursor.execute("SELECT Occupation, COUNT(Occupation) frequency FROM AdultTable GROUP BY Occupation ")
#myresult = cursor.fetchall()
#for x in myresult:
#    print(x)
    
    
con = db.connect('sqladb.sqlite')
with con:
    cur = con.cursor()    
    cur.execute("SELECT Occupation, COUNT(Occupation) Count FROM AdultTable GROUP BY Occupation ")   
    x = from_db_cursor(cur)
    
print(x.get_string(title="Adult Data Group by Occupation"))


+---------------------------------+
|  Adult Data Group by Occupation |
+------------------------+--------+
|       Occupation       | Count  |
+------------------------+--------+
|            ?           |  1843  |
|      Adm-clerical      |  3769  |
|      Armed-Forces      |   9    |
|      Craft-repair      |  4099  |
|     Exec-managerial    |  4066  |
|     Farming-fishing    |  994   |
|    Handlers-cleaners   |  1370  |
|    Machine-op-inspct   |  2002  |
|      Other-service     |  3295  |
|     Priv-house-serv    |  149   |
|     Prof-specialty     |  4140  |
|     Protective-serv    |  649   |
|          Sales         |  3650  |
|      Tech-support      |  928   |
|    Transport-moving    |  1597  |
+------------------------+--------+


In [15]:
##3. Show me the frequency table for education, occupation and relationship, separately

#    a) relationship
#cursor.execute("SELECT Familyrel Relationship, COUNT(Familyrel) count FROM AdultTable GROUP BY Familyrel ")
#myresult = cursor.fetchall()
#for x in myresult:
#    print(x)
    
#for x in myresult:
#    print(x)
    
    
con = db.connect('sqladb.sqlite')
with con:
    cur = con.cursor()    
    cur.execute("SELECT Familyrel Relationship, COUNT(Familyrel) count FROM AdultTable GROUP BY Familyrel ")   
    x = from_db_cursor(cur)
print(x.get_string(title="Adult Data Group by Family Relationship"))


+---------------------------------------+
| Adult Data Group by Family Relationship |
+---------------------------+-----------+
|        Relationship       |   count   |
+---------------------------+-----------+
|           Husband         |   13193   |
|        Not-in-family      |    8304   |
|       Other-relative      |    981    |
|          Own-child        |    5068   |
|          Unmarried        |    3446   |
|            Wife           |    1568   |
+---------------------------+-----------+


In [16]:
# 4) Are there any people who are married, working in private sector and having a masters degree 

#cursor.execute(" SELECT  Maritalstatus, COUNT(Maritalstatus)  FROM AdultTable where ( Maritalstatus IN (' Married-civ-spouse', ' Married-spouse-absent' , ' Married-AF-spouse') AND Workclass = ' Private'  AND Education =  ' Masters'  ) GROUP BY Maritalstatus , Workclass, Education ")
#myresult = cursor.fetchall()
#for x in myresult:
#    print(x)

        
con = db.connect('sqladb.sqlite')
with con:
    cur = con.cursor()    
    cur.execute("SELECT  Maritalstatus AS 'Marital Status', COUNT(Maritalstatus) Count  FROM AdultTable where ( Maritalstatus IN (' Married-civ-spouse', ' Married-spouse-absent' , ' Married-AF-spouse') AND Workclass = ' Private'  AND Education =  ' Masters'  ) GROUP BY Maritalstatus , Workclass, Education ")   
    x = from_db_cursor(cur)
print(x.get_string(title="Adult Data Group by Marital Status"))

+-------------------------------------+
|  Adult Data Group by Marital Status |
+----------------------------+--------+
|       Marital Status       | Count  |
+----------------------------+--------+
|     Married-civ-spouse     |  531   |
|    Married-spouse-absent   |   9    |
+----------------------------+--------+


In [17]:
# 5) Average, minimum and maximum age group for people working in different sectors

#cursor.execute(" SELECT workclass as sector, AVG(Age) averageAge, MIN(Age) minAge, MAX(Age) maxAge  FROM AdultTable GROUP BY workclass ")
#myresult = cursor.fetchall()
#for x in myresult:
#    print(x)
    
con = db.connect('sqladb.sqlite')
with con:
    cur = con.cursor()    
    cur.execute("SELECT workclass AS Sector , AVG(Age) 'Average Age', MIN(Age) 'Min Age', MAX(Age) 'Max Age' FROM AdultTable GROUP BY workclass ")   
    x = from_db_cursor(cur)
print(x.get_string(title="Adult Age Data Group by Sector"))

+------------------------------------------------------------+
|               Adult Age Data Group by Sector               |
+-------------------+--------------------+---------+---------+
|       Sector      |    Average Age     | Min Age | Max Age |
+-------------------+--------------------+---------+---------+
|          ?        | 40.960239651416124 |    17   |    90   |
|     Federal-gov   |     42.590625      |    17   |    90   |
|      Local-gov    | 41.75107501194458  |    17   |    90   |
|    Never-worked   | 20.571428571428573 |    17   |    30   |
|       Private     | 36.797585477617204 |    17   |    90   |
|    Self-emp-inc   | 46.01702508960574  |    17   |    84   |
|  Self-emp-not-inc | 44.96969696969697  |    17   |    90   |
|      State-gov    | 39.43639167309175  |    17   |    81   |
|     Without-pay   | 47.785714285714285 |    19   |    72   |
+-------------------+--------------------+---------+---------+


In [18]:
#6. Calculate age distribution by country
#cursor.execute(" SELECT Nativecountry AS COUNTRY, age,COUNT(age) FROM AdultTable GROUP BY Nativecountry ORDER BY Nativecountry, age ASC")
#myresult = cursor.fetchall()
#for x in myresult:
#    print(x)

con = db.connect('sqladb.sqlite')
with con:
    cur = con.cursor()    
    cur.execute("SELECT Nativecountry AS Country, Age AS AGE,COUNT(age) Count FROM AdultTable GROUP BY Nativecountry ORDER BY Nativecountry, Age  ASC ")   
    x = from_db_cursor(cur)
    df = x
print(x.get_string(title="Adult Age Data Group by Native Country"))


+-------------------------------------------+
|   Adult Age Data Group by Native Country  |
+-----------------------------+-----+-------+
|           Country           | AGE | Count |
+-----------------------------+-----+-------+
|               ?             |  40 |  583  |
|           Cambodia          |  42 |   19  |
|            Canada           |  56 |  121  |
|             China           |  51 |   75  |
|           Columbia          |  75 |   59  |
|             Cuba            |  28 |   95  |
|      Dominican-Republic     |  23 |   70  |
|            Ecuador          |  50 |   28  |
|          El-Salvador        |  23 |  106  |
|            England          |  48 |   90  |
|            France           |  58 |   29  |
|            Germany          |  30 |  137  |
|            Greece           |  22 |   29  |
|           Guatemala         |  37 |   64  |
|             Haiti           |  19 |   44  |
|      Holand-Netherlands     |  32 |   1   |
|           Honduras          |  4

In [19]:
con = db.connect('sqladb.sqlite')
with con:
    cur = con.cursor()    
    cur.execute("SELECT Nativecountry AS Country,count(Age) maxc FROM AdultTable WHERE Nativecountry !=' ?'  GROUP BY Nativecountry, Age ORDER BY Nativecountry, Age  ASC ")   
    x = from_db_cursor(cur)
    df = x
print(x.get_string(title="Adult Age Data Group by Native Country"))


+-----------------------------------------+
|  Adult Age Data Group by Native Country |
+---------------------------------+-------+
|             Country             |  maxc |
+---------------------------------+-------+
|             Cambodia            |   1   |
|             Cambodia            |   1   |
|             Cambodia            |   2   |
|             Cambodia            |   1   |
|             Cambodia            |   1   |
|             Cambodia            |   1   |
|             Cambodia            |   1   |
|             Cambodia            |   1   |
|             Cambodia            |   2   |
|             Cambodia            |   2   |
|             Cambodia            |   1   |
|             Cambodia            |   1   |
|             Cambodia            |   1   |
|             Cambodia            |   1   |
|             Cambodia            |   1   |
|             Cambodia            |   1   |
|              Canada             |   2   |
|              Canada           

In [20]:
#7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'
#cursor.execute(" SELECT Age, Workclass,Capitalgain,Capitalolss, (Capitalgain - Capitalolss) AS NetCapitalGain  FROM AdultTable ")
#myresult = cursor.fetchall()
#for x in myresult:
#    print(x)
#con.close()

con = db.connect('sqladb.sqlite')
with con:
    cur = con.cursor()    
    cur.execute("SELECT Age, Workclass AS Sector,Capitalgain AS 'Capital Gain',Capitalolss AS 'Capital Loss', (Capitalgain - Capitalolss) AS 'Net Capital Gain'  FROM AdultTable WHERE (Workclass  != ' ?' AND (Capitalgain>0  OR Capitalolss >0))  ORDER BY Workclass, Age LIMIT 50 ")   
    x = from_db_cursor(cur)
print(x.get_string(title="Net Capital Gain Calculation"))


+---------------------------------------------------------------------+
|                     Net Capital Gain Calculation                    |
+-----+--------------+--------------+--------------+------------------+
| Age |    Sector    | Capital Gain | Capital Loss | Net Capital Gain |
+-----+--------------+--------------+--------------+------------------+
|  17 |  Federal-gov |      0       |     1602     |      -1602       |
|  22 |  Federal-gov |      0       |     1902     |      -1902       |
|  24 |  Federal-gov |      0       |     1741     |      -1741       |
|  24 |  Federal-gov |      0       |     1974     |      -1974       |
|  26 |  Federal-gov |     3325     |      0       |       3325       |
|  27 |  Federal-gov |      0       |     1980     |      -1980       |
|  27 |  Federal-gov |     4865     |      0       |       4865       |
|  27 |  Federal-gov |     4064     |      0       |       4064       |
|  28 |  Federal-gov |     3887     |      0       |       3887 

In [21]:

# # Create your connection.
# con = db.connect('sqladb.sqlite')

# df = pd.read_sql_query("SELECT Nativecountry, Age FROM AdultTable WHERE Nativecountry != ' ?' GROUP BY Nativecountry, Age ", con)
# df1 = pd.read_sql_query("SELECT Nativecountry AS Country, Age AS AGE,COUNT(Age) Count FROM AdultTable WHERE Nativecountry != ' ?' GROUP BY Nativecountry ORDER BY Nativecountry, Age  ASC ",con)   

# num_bins = 9

# n, bins, patches = plt.hist(df.Age, num_bins, facecolor='blue', alpha=0.5)
# plt.title('Calculate age distribution ')
# plt.xlabel('Age Bin')
# plt.ylabel('Age Count')
# plt.grid(axis='y', alpha=0.75)
# plt.show()

# #df.groupby('Nativecountry').Age.hist()

 


In [22]:
con = db.connect('sqladb.sqlite')
with con:
    cur = con.cursor()    
    cur.execute(" DELETE FROM  AdultTable ")   
    x = from_db_cursor(cur)
print(x)


None
